## Assignment 8

In this assignment, the Credit Card Fraud Detection dataset is used that can be found on [kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In this workbook a classification task to fraudulent and non fraudelent users is done ising support vector machines (SVMs).  
Also techniques to handle imbalanced data are implemented.

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [1]:
# if needed install packages uncommenting and executing the following commands
# !pip install imblearn

In [8]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import os 

os.chdir('C:/Users/anast/OneDrive/Desktop/MSc/MachineLearning/Assignments/Asgmt8_SVM/')

In [3]:
data_file = 'creditcard.csv'

data = pd.read_csv(data_file)

**Scaling**  
Time and amount variables need to be scaled. The rest of the variables (the PCs) are already scaled 


In [4]:
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['Time'] = scaler.fit_transform(data['Time'].values.reshape(-1,1))

In [5]:
X = data.drop(columns='Class')
y = data['Class']

The target variable is extremely imbalanced. Only 0.17% out of all the transactions of the dataset are fraudulent. This problem, may lead any model to overfit the non-fraudulent examples, being unable to recognise fraud. There are different ways to handle such issues.  
Here I will experiment with:
* Undersampling
* Oversampling
* Combination of both

Some nice guides can be found on [DataCamp](https://www.datacamp.com/community/tutorials/diving-deep-imbalanced-data?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034349&utm_targetid=aud-390929969673:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=9061579&gclid=CjwKCAiAq8f-BRBtEiwAGr3Dgc65y799jXfSyX1UAugegeLHUDk7lb6izpB-coR1udmOQvHoN76s2xoCpg8QAvD_BwE), [KDnuggets](https://www.kdnuggets.com/2020/01/5-most-useful-techniques-handle-imbalanced-datasets.html) and [Machine Learning Mastery](https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/).

### Under-sampling

**Random under-sampling** 
 
In random under-sampling, the only a subset of the majority class examples is retained and all the observation of the minority class are retained.

**Pros**: Improve the runtime of the model by reducing the number of training data samples when the training set is gigantic.   
**Cons**: There is high risk of information loss as only a small subset of the majority class training examples is used.


In [6]:
data_fraud = data[data['Class']==1]
data_no_fraud = data[data['Class']==0]

fraud_count = data_fraud['Class'].count()

# undersample majority class
data_no_fraud = resample(data_no_fraud, replace=False, n_samples=int(fraud_count*1.75), random_state=909)

data_undersampled = pd.concat([data_fraud, data_no_fraud])

**Tomek Links**  

Undersampling can also be achieved using [Tomek links](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.TomekLinks.html). Tomek links are pairs of examples of opposite classes in close vicinity. It is a fairly expensive algorithm since it has to compute pairwise distances between all examples. After this calculation, the majority elements from the Tomek link are removed, thus providing a better decision boundary for a classifier. Samples from the majorith, the minority or both classes can be removed.  
Undersampling can also be performed on the resulting dataset as discussed [here](https://www.hilarispublisher.com/open-access/classification-of-imbalance-data-using-tomek-link-tlink-combined-with-random-undersampling-rus-as-a-data-reduction-method-2229-8711-S1111.pdf).

In [7]:
tl = TomekLinks(sampling_strategy='majority')

X_tl, y_tl = tl.fit_resample(X,y)

### Over-sampling

**SMOTE**

Synthetic Minority Oversampling Technique (SMOTE) is an oversampling technique where the synthetic samples are generated for the minority class. This algorithm helps to overcome the overfitting problem posed by random oversampling. It focuses on the feature space to generate new instances with the help of interpolation between the positive instances that lie together.

Specifically, a random example from the minority class is first chosen. Then k of the nearest neighbors for that example are found (typically k=5). A randomly selected neighbor is chosen and a synthetic example is created at a randomly selected point between the two examples in feature space.

In [9]:
sm = SMOTE(sampling_strategy=0.3, random_state=133, n_jobs=-1)

X_sm, y_sm = sm.fit_resample(X,y)